# To run only on local not ssh

In [1]:
import scanpy as sc
import napari
import numpy as np
from matplotlib import cm
from matplotlib.colors import to_hex

# Load data
adata = sc.read_h5ad("/Users/mounim/Downloads/bins_IMMUNEX001.h5ad") 
coords = adata.obsm["spatial"]
coords = coords[:, [1, 0]]  # switch x and y

# Image and scalefactor
sample_id = list(adata.uns["spatial"].keys())[0]
he_img = adata.uns["spatial"][sample_id]["images"]["hires"]
scale = adata.uns["spatial"][sample_id]["scalefactors"]["tissue_hires_scalef"]

# Convert to pixel coordinates for the image
coords_pixels = coords * scale


# Convert image to uint8 if needed
if he_img.max() <= 1.0:
    he_img = (he_img * 255).astype(np.uint8)


In [2]:
from skimage.io import imread

he_fullres = imread("/Users/mounim/Documents/IMMUNEX_data/IMAGE/HE_nanozoomer_tif/IMMUNEX001_Visium_HE_x40_z0.tif")
# viewer.add_image(he_fullres, name="H&E (Fullres)", rgb=True)
print("Image shape:", he_fullres.shape)

scale_factor_y = he_fullres.shape[0] / he_img.shape[0]
scale_factor_x = he_fullres.shape[1] / he_img.shape[1]

print("Scale factors:", scale_factor_y, scale_factor_x)


Image shape: (91904, 119040, 3)
Scale factors: 19.841105354058723 19.84


In [3]:
coords_hd = coords_pixels.copy()
coords_hd[:, 0] *= scale_factor_y  # y
coords_hd[:, 1] *= scale_factor_x  # x

In [8]:
from skimage.transform import resize

# Viewer
viewer = napari.Viewer()

In [9]:
# viewer.add_image(he_fullres, name="H&E", rgb=True)


In [ ]:
features = adata.obs.loc[adata.obs.index[:len(coords_hd)], :].copy()

# Add points (corrected coords)
viewer.add_points(
    coords_hd,
    name="Spots",
    size=10,  # size now corresponds to pixels
    face_color='red',
    blending='translucent'
)


# # Optional labels
# if 'labels_he_expanded' in adata.obs.columns:
#     labels = adata.obs['labels_he_expanded'].values.astype(int)
# elif 'labels_he' in adata.obs.columns:
#     labels = adata.obs['labels_he'].values.astype(int)
# else:
#     labels = None

# if labels is not None:
#     viewer.layers["Spots"].features = {"celltype": labels}
#     viewer.layers["Spots"].face_color = "celltype"
#     color_cycle = [to_hex(c) for c in cm.get_cmap('tab20').colors]
#     viewer.layers["Spots"].face_color_cycle = color_cycle




<Points layer 'Spots' at 0x107a2e1a0>

In [ ]:
from shapely.geometry import Polygon
import numpy as np
from skimage.measure import find_contours
from skimage.transform import AffineTransform
import napari

def is_valid_polygon(poly):
    if len(poly) < 3:
        return False
    try:
        p = Polygon(poly)
        return p.is_valid and p.area > 1e-3
    except:
        return False

# Load label image
labels = np.load("/Users/mounim/Downloads/he_segmentation_IMMUNEX001.npy")

# Extract contours
raw_shapes = [np.fliplr(c) for c in find_contours(labels, level=0.5)]
filtered_shapes = [s for s in raw_shapes if is_valid_polygon(s)]

# Apply affine transform (based on coords_hd range and label size)
scale = (
    coords_hd[:, 0].max() / labels.shape[1],
    coords_hd[:, 1].max() / labels.shape[0]
)
tform = AffineTransform(scale=scale)
aligned_shapes = [tform(s) for s in filtered_shapes]

# Final filter on transformed polygons
aligned_shapes = [s for s in aligned_shapes if is_valid_polygon(s)]

viewer.add_shapes(
    aligned_shapes,
    face_color='none',
    edge_color='red',
    edge_width=1.0,
    shape_type='path',  # <-- Draws only the border, NO triangulation!
    name='Aligned Segmentation Borders'
)
napari.run()


/opt/anaconda3/envs/napari_env/lib/python3.10/site-packages/skimage/transform/_geometric.py:679: RuntimeWarning: divide by zero encountered in matmul
  dst = src @ matrix.T
/opt/anaconda3/envs/napari_env/lib/python3.10/site-packages/skimage/transform/_geometric.py:679: RuntimeWarning: overflow encountered in matmul
  dst = src @ matrix.T
/opt/anaconda3/envs/napari_env/lib/python3.10/site-packages/skimage/transform/_geometric.py:679: RuntimeWarning: invalid value encountered in matmul
  dst = src @ matrix.T
/opt/anaconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/utils/color_transformations.py:42: UserWarning: The provided face_color parameter contained illegal values, resetting all face_color values to white.
  warnings.warn(


In [ ]:
import pandas as pd
# Prepare color cycle for categorical
color_cycle = [to_hex(c) for c in cm.get_cmap('Set1').colors]

# Columns to add as separate layers
color_columns = ['n_counts_adjusted', 'labels_he', 'labels_he_expanded', 'labels_gex', 'labels_joint', 'labels_joint_source']
color_columns = ['labels_joint', 'labels_joint_source']

# Add one layer per column
for col in color_columns:
    data = features.copy()

    # Prepare face color
    if pd.api.types.is_numeric_dtype(data[col]):
        face_color = data[col].values
        face_color_mode = 'colormap'
        colormap = 'viridis'
        categorical = False
    else:
        data[col] = pd.Categorical(data[col])
        face_color = data[col]
        face_color_mode = 'cycle'
        colormap = None
        categorical = True

            
        # Get unique categories
        unique_cats = data[col].cat.categories
        n_colors = len(unique_cats)

        # Get corresponding colors from your Set1 color cycle
        legend_colors = color_cycle[:n_colors]

        # Create legend positions (top-left corner, staggered)
        legend_coords = []
        legend_labels = []
        for i, cat in enumerate(unique_cats):
            y = 50 + i * 25  # vertical spacing
            x = 50           # fixed horizontal position
            legend_coords.append([y, x])
            legend_labels.append(str(cat))

        legend_coords = np.array(legend_coords)

        # Add legend points (fake layer)
        legend_layer = viewer.add_points(
            legend_coords,
            name=f"{layer_name}_legend",
            face_color=legend_colors,
            size=10,
            blending='opaque',
            features={"label": legend_labels}
        )

        legend_layer.text = {
            "string": "{label}",
            "anchor": "center",             # valid value
            "translation": [15, 0],         # shift text right
            "color": "white",
            "size": 10,
        }


    layer_name = f"Spots_{col}"
    viewer.add_points(
        coords_hd,
        name=layer_name,
        features={col: face_color},
        face_color=col,
        face_color_cycle=color_cycle if categorical else None,
        size=10,
        blending='translucent'
    )
